# Imaging utilities
Andrey Tremba  
11.2021

Usage:
```python
# 'ipynb as module' loader
import ipynb_loader
# import ipynb itself
import image_utils
# or import image_utils as imut

fig_3d = plt.figure()
ax_3d = fig_3d.add_subplot(projection='3d')

... # do something, e.g. ax_3d_equal.scatter(X, Y, Z)

... # set p
square_XYZ = image_utils.get_rotated_square(p)
ax_3d_square.plot_surface(*square_XYZ, alpha=0.3)

... # finally set axes equal

image_utils.set_axes3d_equal(ax_3d)

```

See also `image-utils-demo.ipynb`

Effect and sources:

1. Making axes in 3D plot equal. Shall be run after all plotting. <https://stackoverflow.com/questions/13685386/matplotlib-equal-unit-length-with-equal-aspect-ratio-z-axis-is-not-equal-to>

In [ ]:
import numpy as np

In [ ]:
# https://stackoverflow.com/questions/13685386/matplotlib-equal-unit-length-with-equal-aspect-ratio-z-axis-is-not-equal-to
def set_axes3d_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    
    Upd: aspect correction added (fix default vertical distortion).
    '''
    ax.set_box_aspect([3, 3, 3]) # by default the _visualization_ box aspect is 4:4:3 

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

In [ ]:
def get_rotated_square(p, square_size=0.75, debug=False):
    """Plot a square, located at the end of vector p and orthogonal to it"""
    # make square centered at (0, 0, 1) parallel to XY axes
    square_size_doubled = 2 * square_size
    n_points = 2
    xx, yy = np.meshgrid(np.linspace(-square_size_doubled, square_size_doubled, n_points), \
                         np.linspace(-square_size_doubled, square_size_doubled, n_points))
    zz = np.ones_like(xx)
    # ax_3d.plot_surface(xx, yy, zz)
    if debug:
        print(f'{xx.shape}, {yy.shape}, {zz.shape}')
    z_axis = np.array([0, 0, 1])

    # Make rotation from z-axis to p-aligned axis
    # https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d/897677#897677
    p_unit = p / np.linalg.norm(p)
    d = z_axis + p_unit
    R = 2 * np.atleast_2d(d).T @ np.atleast_2d(d) / (d @ d) - np.eye(3)
    if debug:
        print(f'{R.shape}, {R @ p_unit}')
    mult_1 = lambda x1, x2, x3 : np.linalg.norm(p) * R @ np.array([x1, x2, x3])
    rotate_func = np.vectorize(mult_1, signature='(),(),()->(k)')
    if debug:
        print(np.ravel(xx))
        zipped = tuple(zip(np.ravel(xx), np.ravel(yy), np.ravel(zz)))
        # print(zipped)
        print(f'{len(zipped)}, {zipped[0]}')
    ZZ = rotate_func(np.ravel(xx), np.ravel(yy), np.ravel(zz))
    if debug:
        print(ZZ.shape)
    # return tuple of 3 matrices
    return ZZ[:,0].reshape([n_points,n_points]), ZZ[:,1].reshape([n_points,n_points]), ZZ[:,2].reshape([n_points,n_points])